In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score
)

import mlflow
import mlflow.sklearn


In [14]:
final_df = pd.read_csv("../data/processed/final_with_target.csv")

In [15]:
y = final_df["is_high_risk"]
X = final_df.drop(columns=["CustomerId", "is_high_risk"])

In [16]:
X = pd.get_dummies(X)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=100, random_state=42)
}

mlflow.set_experiment("credit-risk-model")

<Experiment: artifact_location='file:///c:/Users/bless/OneDrive/Desktop/week%205/credit-risk-model/notebooks/mlruns/103085281130307774', creation_time=1751114718298, experiment_id='103085281130307774', last_update_time=1751114718298, lifecycle_stage='active', name='credit-risk-model', tags={}>

In [19]:
for name, model in models.items():
    with mlflow.start_run(run_name=name):
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else preds

        metrics = {
            "accuracy": accuracy_score(y_test, preds),
            "precision": precision_score(y_test, preds),
            "recall": recall_score(y_test, preds),
            "f1_score": f1_score(y_test, preds),
            "roc_auc": roc_auc_score(y_test, proba)
        }

        for key, value in metrics.items():
            mlflow.log_metric(key, value)

        mlflow.sklearn.log_model(model, name)
        print(f"✅ {name} trained and logged to MLflow.")

2025/06/28 15:50:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 15:50:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ LogisticRegression trained and logged to MLflow.


2025/06/28 15:50:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 15:50:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ RandomForest trained and logged to MLflow.


2025/06/28 15:50:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/28 15:50:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ GradientBoosting trained and logged to MLflow.
